In [1]:
import json
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import ccxt
import requests
import time
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from defillama import DefiLlama
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import ccxt
import requests
from datetime import datetime
from defillama import DefiLlama
import json
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import ccxt
import requests
from datetime import datetime
from defillama import DefiLlama

In [2]:
# Now, let's define the database models using SQLAlchemy:

In [2]:

# Define the database engine
engine = create_engine('sqlite:///crypto_data.db')
def create_database():
    engine = create_engine('sqlite:///crypto_data.db')
    Base.metadata.create_all(engine)
    print("Database created successfully!")

In [3]:
Base = declarative_base()

class Token(Base):
    __tablename__ = 'tokens'
    id = Column(Integer, primary_key=True)
    symbol = Column(String)
    name = Column(String)

class HistoricalMarketCap(Base):
    __tablename__ = 'historical_market_caps'
    id = Column(Integer, primary_key=True)
    token_id = Column(Integer, ForeignKey('tokens.id'))
    source = Column(String)
    timestamp = Column(DateTime)
    market_cap = Column(Float)
    token = relationship('Token', backref='historical_market_caps')
    
    
# Define the ExchangePrice table
class ExchangePrice(Base):
    __tablename__ = 'exchange_prices'
    id = Column(Integer, primary_key=True)
    symbol = Column(String)
    exchange = Column(String)
    timestamp = Column(DateTime)
    open_price = Column(Float)
    high_price = Column(Float)
    low_price = Column(Float)
    close_price = Column(Float)

class DeFiLlamaTVL(Base):
    __tablename__ = 'defillama_tvl'
    id = Column(Integer, primary_key=True)
    protocol = Column(String)
    timestamp = Column(DateTime)
    tvl = Column(Float)


    

In [ ]:
#fetch btc market cap from coingecko

In [4]:


def fetch_historical_data(symbol, name, token_id):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 1000,
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'prices' in data and 'market_caps' in data:
            prices = data['prices']
            market_caps = data['market_caps']
            
            historical_market_caps = []
            for timestamp, cap in market_caps:
                timestamp = datetime.fromtimestamp(timestamp / 1000)
                timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                historical_market_caps.append(
                    {
                        'token_id': token_id,
                        'source': 'Coingecko',
                        'timestamp': timestamp,
                        'market_cap': cap
                    }
                )
            
            return historical_market_caps
        else:
            print(f"Failed to fetch historical data for {symbol} from Coingecko.")
    else:
        print(f"Failed to fetch historical data for {symbol}: {response.status_code}")
    
    return None

def insert_data_into_database(data, name, symbol):
    if data:
        engine = create_engine('sqlite:///crypto_data.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        token = session.query(Token).filter(Token.symbol == symbol).first()
        if token is None:
            token = Token(symbol=symbol, name=name)
            session.add(token)
            session.flush()

        existing_timestamps = set(
            session.query(HistoricalMarketCap.timestamp)
            .filter(HistoricalMarketCap.token_id == token.id)
            .all()
        )
        
        new_data = [
            item for item in data
            if item['timestamp'] not in existing_timestamps
        ]
        
        for item in data:
            timestamp = datetime.strptime(item['timestamp'], '%Y-%m-%d %H:%M:%S')
            historical_market_cap = HistoricalMarketCap(token_id=token.id, source=item['source'], timestamp=timestamp, market_cap=item['market_cap'])
            session.add(historical_market_cap)
        
        session.commit()

        print("Data inserted into the database successfully!")
        print("Preview of fetched data:")
        for item in new_data:
            print(f"Token ID: {item['token_id']}, Source: {item['source']}, Timestamp: {item['timestamp']}, Market Cap: {item['market_cap']}")
    else:
        print("No new data to insert into the database.")

def main():
    token_symbols = [{'symbol': 'bitcoin', 'name': 'Bitcoin', 'token_id': 1}]

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        token_id = token['token_id']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, name, token_id)
        insert_data_into_database(historical_data, name, symbol)

if __name__ == '__main__':
    main()


Fetching data for token with symbol: bitcoin
Data inserted into the database successfully!
Preview of fetched data:
Token ID: 1, Source: Coingecko, Timestamp: 2020-09-26 01:00:00, Market Cap: 197616950282.06653
Token ID: 1, Source: Coingecko, Timestamp: 2020-09-27 01:00:00, Market Cap: 198687083452.04324
Token ID: 1, Source: Coingecko, Timestamp: 2020-09-28 01:00:00, Market Cap: 199236216251.43555
Token ID: 1, Source: Coingecko, Timestamp: 2020-09-29 01:00:00, Market Cap: 197720212770.89465
Token ID: 1, Source: Coingecko, Timestamp: 2020-09-30 01:00:00, Market Cap: 200532996047.58585
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-01 01:00:00, Market Cap: 199310441688.83572
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-02 01:00:00, Market Cap: 196617081790.5188
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-03 01:00:00, Market Cap: 195660035179.73257
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-04 01:00:00, Market Cap: 195188285650.34625
Token ID: 1, Source: Coingecko

In [ ]:
#fetch eth market cap from coingecko

In [5]:


def fetch_historical_data(symbol, name, token_id):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 1000,
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'prices' in data and 'market_caps' in data:
            prices = data['prices']
            market_caps = data['market_caps']
            
            historical_market_caps = []
            for timestamp, cap in market_caps:
                timestamp = datetime.fromtimestamp(timestamp / 1000)
                timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                historical_market_caps.append(
                    {
                        'token_id': token_id,
                        'source': 'Coingecko',
                        'timestamp': timestamp,
                        'market_cap': cap
                    }
                )
            
            return historical_market_caps
        else:
            print(f"Failed to fetch historical data for {symbol} from Coingecko.")
    else:
        print(f"Failed to fetch historical data for {symbol}: {response.status_code}")
    
    return None

def insert_data_into_database(data, name, symbol):
    if data:
        engine = create_engine('sqlite:///crypto_data.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        token = session.query(Token).filter(Token.symbol == symbol).first()
        if token is None:
            token = Token(symbol=symbol, name=name)
            session.add(token)
            session.flush()

        existing_timestamps = set(
            session.query(HistoricalMarketCap.timestamp)
            .filter(HistoricalMarketCap.token_id == token.id)
            .all()
        )
        
        new_data = [
            item for item in data
            if item['timestamp'] not in existing_timestamps
        ]
        
        for item in data:
            timestamp = datetime.strptime(item['timestamp'], '%Y-%m-%d %H:%M:%S')
            historical_market_cap = HistoricalMarketCap(token_id=token.id, source=item['source'], timestamp=timestamp, market_cap=item['market_cap'])
            session.add(historical_market_cap)
        
        session.commit()

        print("Data inserted into the database successfully!")
        print("Preview of fetched data:")
        for item in new_data:
            print(f"Token ID: {item['token_id']}, Source: {item['source']}, Timestamp: {item['timestamp']}, Market Cap: {item['market_cap']}")
    else:
        print("No new data to insert into the database.")

def main():
    token_symbols = [{'symbol': 'ethereum', 'name': 'Ethereum', 'token_id': 2}]

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        token_id = token['token_id']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, name, token_id)
        insert_data_into_database(historical_data, name, symbol)

if __name__ == '__main__':
    main()

Fetching data for token with symbol: ethereum
Data inserted into the database successfully!
Preview of fetched data:
Token ID: 2, Source: Coingecko, Timestamp: 2020-09-26 01:00:00, Market Cap: 39598389061.56603
Token ID: 2, Source: Coingecko, Timestamp: 2020-09-27 01:00:00, Market Cap: 39946698187.61045
Token ID: 2, Source: Coingecko, Timestamp: 2020-09-28 01:00:00, Market Cap: 40273903426.08249
Token ID: 2, Source: Coingecko, Timestamp: 2020-09-29 01:00:00, Market Cap: 39794887269.461235
Token ID: 2, Source: Coingecko, Timestamp: 2020-09-30 01:00:00, Market Cap: 40574256067.82458
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-01 01:00:00, Market Cap: 40586538615.954575
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-02 01:00:00, Market Cap: 39782583382.89109
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-03 01:00:00, Market Cap: 39005631807.605095
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-04 01:00:00, Market Cap: 39098719406.98244
Token ID: 2, Source: Coingecko, Ti

In [ ]:
#binance price data

In [6]:
# Fetch historical data for a specific symbol
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = json.loads(response.text)
        return data
    else:
        print('Error occurred. Status Code:', response.status_code)
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for entry in data:
        timestamp = entry[0]
        readable_timestamp = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(entry[1])
        high_price = float(entry[2])
        low_price = float(entry[3])
        close_price = float(entry[4])
        exchange = 'Binance'  # Set the exchange name as 'Binance'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='ETHUSDT' OR symbol='BTCUSDT'")
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    token_symbols = [
        {'symbol': 'BTCUSDT', 'name': 'Bitcoin'},
        {'symbol': 'ETHUSDT', 'name': 'Ethereum'},
        # Add more tokens here
    ]

    interval = '1d'  # 1 day interval
    limit = 30  # Retrieve 30 days of historical data

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, interval, limit)
        if historical_data:
            insert_data_into_database(historical_data, symbol)
        else:
            print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()

Fetching data for token with symbol: BTCUSDT
Data inserted into the database successfully!
Fetching data for token with symbol: ETHUSDT
Data inserted into the database successfully!
Retrieved data from the database:
(1, 'BTCUSDT', 'Binance', '2023-05-24 01:00:00', 27219.61, 27219.61, 26080.5, 26329.01)
(2, 'BTCUSDT', 'Binance', '2023-05-25 01:00:00', 26329.0, 26631.98, 25871.89, 26473.79)
(3, 'BTCUSDT', 'Binance', '2023-05-26 01:00:00', 26473.8, 26932.16, 26327.24, 26705.92)
(4, 'BTCUSDT', 'Binance', '2023-05-27 01:00:00', 26705.93, 26895.0, 26551.0, 26854.27)
(5, 'BTCUSDT', 'Binance', '2023-05-28 01:00:00', 26854.28, 28261.32, 26764.36, 28065.0)
(6, 'BTCUSDT', 'Binance', '2023-05-29 01:00:00', 28065.01, 28447.14, 27524.6, 27736.4)
(7, 'BTCUSDT', 'Binance', '2023-05-30 01:00:00', 27736.39, 28038.59, 27554.0, 27694.4)
(8, 'BTCUSDT', 'Binance', '2023-05-31 01:00:00', 27694.39, 27835.51, 26839.01, 27210.35)
(9, 'BTCUSDT', 'Binance', '2023-06-01 01:00:00', 27210.36, 27350.0, 26605.05, 2681

In [7]:
# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])
        exchange = 'Kraken'  # Set the exchange name as 'Kraken'

        # Modify the symbol to BTCUSDT
        symbol = 'BTCUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='BTCUSDT'")  # Change the symbol to 'BTCUSDT' for BTC to USDT pair
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'XXBTZUSD'  # BTC to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database:
(1, 'BTCUSDT', 'Binance', '2023-05-24 01:00:00', 27219.61, 27219.61, 26080.5, 26329.01)
(2, 'BTCUSDT', 'Binance', '2023-05-25 01:00:00', 26329.0, 26631.98, 25871.89, 26473.79)
(3, 'BTCUSDT', 'Binance', '2023-05-26 01:00:00', 26473.8, 26932.16, 26327.24, 26705.92)
(4, 'BTCUSDT', 'Binance', '2023-05-27 01:00:00', 26705.93, 26895.0, 26551.0, 26854.27)
(5, 'BTCUSDT', 'Binance', '2023-05-28 01:00:00', 26854.28, 28261.32, 26764.36, 28065.0)
(6, 'BTCUSDT', 'Binance', '2023-05-29 01:00:00', 28065.01, 28447.14, 27524.6, 27736.4)
(7, 'BTCUSDT', 'Binance', '2023-05-30 01:00:00', 27736.39, 28038.59, 27554.0, 27694.4)
(8, 'BTCUSDT', 'Binance', '2023-05-31 01:00:00', 27694.39, 27835.51, 26839.01, 27210.35)
(9, 'BTCUSDT', 'Binance', '2023-06-01 01:00:00', 27210.36, 27350.0, 26605.05, 26817.93)
(10, 'BTCUSDT', 'Binance', '2023-06-02 01:00:00', 26817.93, 27300.0, 26505.0, 27242.59)
(11, 'BTCUSDT', 'Binance', '2023-06-03 01:

In [ ]:
#subsequent btcusd kraken code 

In [8]:
# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])
        exchange = 'Kraken'  # Set the exchange name as 'Kraken'

        # Modify the symbol to BTCUSDT
        symbol = 'BTCUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='BTCUSDT'")  # Change the symbol to 'BTCUSDT' for BTC to USDT pair
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'XXBTZUSD'  # BTC to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database:
(1, 'BTCUSDT', 'Binance', '2023-05-24 01:00:00', 27219.61, 27219.61, 26080.5, 26329.01)
(2, 'BTCUSDT', 'Binance', '2023-05-25 01:00:00', 26329.0, 26631.98, 25871.89, 26473.79)
(3, 'BTCUSDT', 'Binance', '2023-05-26 01:00:00', 26473.8, 26932.16, 26327.24, 26705.92)
(4, 'BTCUSDT', 'Binance', '2023-05-27 01:00:00', 26705.93, 26895.0, 26551.0, 26854.27)
(5, 'BTCUSDT', 'Binance', '2023-05-28 01:00:00', 26854.28, 28261.32, 26764.36, 28065.0)
(6, 'BTCUSDT', 'Binance', '2023-05-29 01:00:00', 28065.01, 28447.14, 27524.6, 27736.4)
(7, 'BTCUSDT', 'Binance', '2023-05-30 01:00:00', 27736.39, 28038.59, 27554.0, 27694.4)
(8, 'BTCUSDT', 'Binance', '2023-05-31 01:00:00', 27694.39, 27835.51, 26839.01, 27210.35)
(9, 'BTCUSDT', 'Binance', '2023-06-01 01:00:00', 27210.36, 27350.0, 26605.05, 26817.93)
(10, 'BTCUSDT', 'Binance', '2023-06-02 01:00:00', 26817.93, 27300.0, 26505.0, 27242.59)
(11, 'BTCUSDT', 'Binance', '2023-06-03 01:

In [9]:

# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])
        exchange = 'Kraken'  # Set the exchange name as 'Kraken'

        # Modify the symbol to ETHUSDT
        symbol = 'ETHUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='ETHUSDT'")  # Change the symbol to 'ETHUSDT' for ETH to USDT pair
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'XETHZUSD'  # ETH to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database:
(31, 'ETHUSDT', 'Binance', '2023-05-24 01:00:00', 1854.06, 1854.09, 1777.6, 1799.89)
(32, 'ETHUSDT', 'Binance', '2023-05-25 01:00:00', 1799.89, 1818.81, 1761.41, 1805.6)
(33, 'ETHUSDT', 'Binance', '2023-05-26 01:00:00', 1805.61, 1838.2, 1796.99, 1827.79)
(34, 'ETHUSDT', 'Binance', '2023-05-27 01:00:00', 1827.8, 1836.12, 1812.18, 1829.89)
(35, 'ETHUSDT', 'Binance', '2023-05-28 01:00:00', 1829.89, 1918.65, 1823.6, 1908.64)
(36, 'ETHUSDT', 'Binance', '2023-05-29 01:00:00', 1908.64, 1927.62, 1873.65, 1892.6)
(37, 'ETHUSDT', 'Binance', '2023-05-30 01:00:00', 1892.6, 1917.0, 1881.19, 1900.59)
(38, 'ETHUSDT', 'Binance', '2023-05-31 01:00:00', 1900.58, 1907.13, 1846.95, 1873.63)
(39, 'ETHUSDT', 'Binance', '2023-06-01 01:00:00', 1873.63, 1889.2, 1840.0, 1861.78)
(40, 'ETHUSDT', 'Binance', '2023-06-02 01:00:00', 1861.78, 1910.0, 1847.51, 1906.68)
(41, 'ETHUSDT', 'Binance', '2023-06-03 01:00:00', 1906.69, 1907.92, 188

In [ ]:
#coinbase 

In [10]:

# Fetch historical data for a specific symbol
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])
        exchange = 'Coinbase'  # Set the exchange name as 'Coinbase'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='ETHUSDT'")
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'ETH-USD'  # ETH to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, 'ETHUSDT')
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()


Data inserted into the database successfully!
Retrieved data from the database:
(31, 'ETHUSDT', 'Binance', '2023-05-24 01:00:00', 1854.06, 1854.09, 1777.6, 1799.89)
(32, 'ETHUSDT', 'Binance', '2023-05-25 01:00:00', 1799.89, 1818.81, 1761.41, 1805.6)
(33, 'ETHUSDT', 'Binance', '2023-05-26 01:00:00', 1805.61, 1838.2, 1796.99, 1827.79)
(34, 'ETHUSDT', 'Binance', '2023-05-27 01:00:00', 1827.8, 1836.12, 1812.18, 1829.89)
(35, 'ETHUSDT', 'Binance', '2023-05-28 01:00:00', 1829.89, 1918.65, 1823.6, 1908.64)
(36, 'ETHUSDT', 'Binance', '2023-05-29 01:00:00', 1908.64, 1927.62, 1873.65, 1892.6)
(37, 'ETHUSDT', 'Binance', '2023-05-30 01:00:00', 1892.6, 1917.0, 1881.19, 1900.59)
(38, 'ETHUSDT', 'Binance', '2023-05-31 01:00:00', 1900.58, 1907.13, 1846.95, 1873.63)
(39, 'ETHUSDT', 'Binance', '2023-06-01 01:00:00', 1873.63, 1889.2, 1840.0, 1861.78)
(40, 'ETHUSDT', 'Binance', '2023-06-02 01:00:00', 1861.78, 1910.0, 1847.51, 1906.68)
(41, 'ETHUSDT', 'Binance', '2023-06-03 01:00:00', 1906.69, 1907.92, 188

In [11]:


# Fetch historical data for a specific symbol
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])
        exchange = 'Coinbase'  # Set the exchange name as 'Coinbase'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='BTCUSDT'")
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'BTC-USD'  # BTC to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, 'BTCUSDT')
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()


Data inserted into the database successfully!
Retrieved data from the database:
(1, 'BTCUSDT', 'Binance', '2023-05-24 01:00:00', 27219.61, 27219.61, 26080.5, 26329.01)
(2, 'BTCUSDT', 'Binance', '2023-05-25 01:00:00', 26329.0, 26631.98, 25871.89, 26473.79)
(3, 'BTCUSDT', 'Binance', '2023-05-26 01:00:00', 26473.8, 26932.16, 26327.24, 26705.92)
(4, 'BTCUSDT', 'Binance', '2023-05-27 01:00:00', 26705.93, 26895.0, 26551.0, 26854.27)
(5, 'BTCUSDT', 'Binance', '2023-05-28 01:00:00', 26854.28, 28261.32, 26764.36, 28065.0)
(6, 'BTCUSDT', 'Binance', '2023-05-29 01:00:00', 28065.01, 28447.14, 27524.6, 27736.4)
(7, 'BTCUSDT', 'Binance', '2023-05-30 01:00:00', 27736.39, 28038.59, 27554.0, 27694.4)
(8, 'BTCUSDT', 'Binance', '2023-05-31 01:00:00', 27694.39, 27835.51, 26839.01, 27210.35)
(9, 'BTCUSDT', 'Binance', '2023-06-01 01:00:00', 27210.36, 27350.0, 26605.05, 26817.93)
(10, 'BTCUSDT', 'Binance', '2023-06-02 01:00:00', 26817.93, 27300.0, 26505.0, 27242.59)
(11, 'BTCUSDT', 'Binance', '2023-06-03 01:

In [14]:


# Fetch historical data for a specific symbol
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=?", (symbol,))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])
        exchange = 'Coinbase'  # Set the exchange name as 'Coinbase'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database
def retrieve_data_from_database():
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE symbol='ETHUSDT'")
    rows = c.fetchall()

    print('Retrieved data from the database:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'ETH-USD'  # BTC to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, 'ETHUSDT')
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database()

if __name__ == '__main__':
    main()


Data inserted into the database successfully!
Retrieved data from the database:
(31, 'ETHUSDT', 'Binance', '2023-05-24 01:00:00', 1854.06, 1854.09, 1777.6, 1799.89)
(32, 'ETHUSDT', 'Binance', '2023-05-25 01:00:00', 1799.89, 1818.81, 1761.41, 1805.6)
(33, 'ETHUSDT', 'Binance', '2023-05-26 01:00:00', 1805.61, 1838.2, 1796.99, 1827.79)
(34, 'ETHUSDT', 'Binance', '2023-05-27 01:00:00', 1827.8, 1836.12, 1812.18, 1829.89)
(35, 'ETHUSDT', 'Binance', '2023-05-28 01:00:00', 1829.89, 1918.65, 1823.6, 1908.64)
(36, 'ETHUSDT', 'Binance', '2023-05-29 01:00:00', 1908.64, 1927.62, 1873.65, 1892.6)
(37, 'ETHUSDT', 'Binance', '2023-05-30 01:00:00', 1892.6, 1917.0, 1881.19, 1900.59)
(38, 'ETHUSDT', 'Binance', '2023-05-31 01:00:00', 1900.58, 1907.13, 1846.95, 1873.63)
(39, 'ETHUSDT', 'Binance', '2023-06-01 01:00:00', 1873.63, 1889.2, 1840.0, 1861.78)
(40, 'ETHUSDT', 'Binance', '2023-06-02 01:00:00', 1861.78, 1910.0, 1847.51, 1906.68)
(41, 'ETHUSDT', 'Binance', '2023-06-03 01:00:00', 1906.69, 1907.92, 188

In [ ]:
#bittrex ethusdt

In [13]:

def fetch_data(symbol):
    url = f"https://api.bittrex.com/v3/markets/{symbol}/candles/DAY_1/recent"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from Bittrex. Status code: {response.status_code}")
        return None

def save_data_to_database(data, symbol):
    engine = create_engine('sqlite:///crypto_data.db')
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    for candle in data:
        timestamp = datetime.strptime(candle['startsAt'], "%Y-%m-%dT%H:%M:%SZ")
        open_price = float(candle['open'])
        high_price = float(candle['high'])
        low_price = float(candle['low'])
        close_price = float(candle['close'])
        symbol_in_database = 'ETHUSDT'  # Set the symbol as 'ETHUSDT' for consistency
        existing_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                            ExchangePrice.timestamp == timestamp).first()
        if existing_data:
            # Update existing data
            existing_data.exchange = 'Bittrex'
            existing_data.open_price = open_price
            existing_data.high_price = high_price
            existing_data.low_price = low_price
            existing_data.close_price = close_price
        else:
            # Insert new data
            exchange_price = ExchangePrice(
                symbol=symbol_in_database,
                exchange='Bittrex',
                timestamp=timestamp,
                open_price=open_price,
                high_price=high_price,
                low_price=low_price,
                close_price=close_price
            )
            session.add(exchange_price)

    session.commit()
    session.close()
    print('Data saved to the database successfully!')

def retrieve_data_from_database():
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    symbol_in_database = 'ETHUSDT'
    exchange_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database).all()

    print('ETH data from the database:')
    for data in exchange_data:
        print(f"Timestamp: {data.timestamp}, Open: {data.open_price}, High: {data.high_price}, Low: {data.low_price}, Close: {data.close_price}, Exchange: {data.exchange}")

    session.close()

if __name__ == '__main__':
    symbol = 'eth-usdt'  # ETH to USDT pair (Bittrex symbol)
    eth_data = fetch_data(symbol)
    if eth_data:
        save_data_to_database(eth_data, symbol)
    else:
        print('Failed to fetch data from Bittrex. No data to save to the database.')

    retrieve_data_from_database()

Data saved to the database successfully!
ETH data from the database:
Timestamp: 2023-05-24 01:00:00, Open: 1854.06, High: 1854.09, Low: 1777.6, Close: 1799.89, Exchange: Binance
Timestamp: 2023-05-25 01:00:00, Open: 1799.89, High: 1818.81, Low: 1761.41, Close: 1805.6, Exchange: Binance
Timestamp: 2023-05-26 01:00:00, Open: 1805.61, High: 1838.2, Low: 1796.99, Close: 1827.79, Exchange: Binance
Timestamp: 2023-05-27 01:00:00, Open: 1827.8, High: 1836.12, Low: 1812.18, Close: 1829.89, Exchange: Binance
Timestamp: 2023-05-28 01:00:00, Open: 1829.89, High: 1918.65, Low: 1823.6, Close: 1908.64, Exchange: Binance
Timestamp: 2023-05-29 01:00:00, Open: 1908.64, High: 1927.62, Low: 1873.65, Close: 1892.6, Exchange: Binance
Timestamp: 2023-05-30 01:00:00, Open: 1892.6, High: 1917.0, Low: 1881.19, Close: 1900.59, Exchange: Binance
Timestamp: 2023-05-31 01:00:00, Open: 1900.58, High: 1907.13, Low: 1846.95, Close: 1873.63, Exchange: Binance
Timestamp: 2023-06-01 01:00:00, Open: 1873.63, High: 1889.2

In [ ]:
#btcusdt (bittrex)

In [15]:

def fetch_data(symbol):
    url = f"https://api.bittrex.com/v3/markets/{symbol}/candles/DAY_1/recent"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from Bittrex. Status code: {response.status_code}")
        return None

def save_data_to_database(data, symbol):
    engine = create_engine('sqlite:///crypto_data.db')
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    for candle in data:
        timestamp = datetime.strptime(candle['startsAt'], "%Y-%m-%dT%H:%M:%SZ")
        open_price = float(candle['open'])
        high_price = float(candle['high'])
        low_price = float(candle['low'])
        close_price = float(candle['close'])
        symbol_in_database = 'BTCUSDT'  # Set the symbol as 'BTCUSDT' for consistency
        existing_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                            ExchangePrice.timestamp == timestamp).first()
        if existing_data:
            # Update existing data
            existing_data.exchange = 'Bittrex'
            existing_data.open_price = open_price
            existing_data.high_price = high_price
            existing_data.low_price = low_price
            existing_data.close_price = close_price
        else:
            # Insert new data
            exchange_price = ExchangePrice(
                symbol=symbol_in_database,
                exchange='Bittrex',
                timestamp=timestamp,
                open_price=open_price,
                high_price=high_price,
                low_price=low_price,
                close_price=close_price
            )
            session.add(exchange_price)

    session.commit()
    session.close()
    print('Data saved to the database successfully!')

def retrieve_data_from_database():
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    symbol_in_database = 'BTCUSDT'
    exchange_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database).all()

    print('BTC data from the database:')
    for data in exchange_data:
        print(f"Timestamp: {data.timestamp}, Open: {data.open_price}, High: {data.high_price}, Low: {data.low_price}, Close: {data.close_price}, Exchange: {data.exchange}")

    session.close()

if __name__ == '__main__':
    symbol = 'btc-usdt'  # BTC to USDT pair (Bittrex symbol)
    btc_data = fetch_data(symbol)
    if btc_data:
        save_data_to_database(btc_data, symbol)
    else:
        print('Failed to fetch data from Bittrex. No data to save to the database.')

    retrieve_data_from_database()

Data saved to the database successfully!
BTC data from the database:
Timestamp: 2023-05-24 01:00:00, Open: 27219.61, High: 27219.61, Low: 26080.5, Close: 26329.01, Exchange: Binance
Timestamp: 2023-05-25 01:00:00, Open: 26329.0, High: 26631.98, Low: 25871.89, Close: 26473.79, Exchange: Binance
Timestamp: 2023-05-26 01:00:00, Open: 26473.8, High: 26932.16, Low: 26327.24, Close: 26705.92, Exchange: Binance
Timestamp: 2023-05-27 01:00:00, Open: 26705.93, High: 26895.0, Low: 26551.0, Close: 26854.27, Exchange: Binance
Timestamp: 2023-05-28 01:00:00, Open: 26854.28, High: 28261.32, Low: 26764.36, Close: 28065.0, Exchange: Binance
Timestamp: 2023-05-29 01:00:00, Open: 28065.01, High: 28447.14, Low: 27524.6, Close: 27736.4, Exchange: Binance
Timestamp: 2023-05-30 01:00:00, Open: 27736.39, High: 28038.59, Low: 27554.0, Close: 27694.4, Exchange: Binance
Timestamp: 2023-05-31 01:00:00, Open: 27694.39, High: 27835.51, Low: 26839.01, Close: 27210.35, Exchange: Binance
Timestamp: 2023-06-01 01:00:0

In [ ]:
# Fetch historical TVLs from DeFi Llama API

In [ ]:
def fetch_defillama_data():
    defillama = DefiLlama()
    protocols = defillama.get_all_protocols()

    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_tvls = protocol.get('tvl')

        if protocol_tvls:
            print(f"Protocol: {protocol_name}")
            print("Historical TVLs:")

            if isinstance(protocol_tvls, dict):
                protocol_tvls = [(timestamp, tvl) for timestamp, tvl in protocol_tvls.items()]
            elif isinstance(protocol_tvls, (int, float)):
                protocol_tvls = [(datetime.now(), protocol_tvls)]

            for timestamp, tvl in protocol_tvls:
                if isinstance(timestamp, (int, float)):
                    timestamp = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')
                else:
                    timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                print(f"Timestamp: {timestamp}, TVL: {tvl}")
                
                # Create a connection to the database
                conn = sqlite3.connect('defillama.db')
                c = conn.cursor()
                
                # Execute the SQL statement to insert data
                c.execute("INSERT INTO defillama_tvl (protocol, timestamp, tvl) VALUES (?, ?, ?)",
                          (protocol_name, timestamp, tvl))
                
                # Commit the changes to the database
                conn.commit()
                
                # Close the connection
                conn.close()

    preview_defillama_data_from_database()

def preview_defillama_data_from_database():
    # Create a connection to the database
    conn = sqlite3.connect('defillama.db')
    c = conn.cursor()
    
    # Execute the SQL statement to retrieve data
    c.execute("SELECT * FROM defillama_tvl LIMIT 10")
    
    # Fetch the data and print the preview
    tvl_data = c.fetchall()
    
    for entry in tvl_data:
        print(f"Protocol: {entry[1]}, Timestamp: {entry[2]}, TVL: {entry[3]}")
    
    # Close the connection
    conn.close()

# Call the fetch_defillama_data() function to fetch data from DeFiLlama and save it to the database
fetch_defillama_data()

Protocol: Binance CEX
Historical TVLs:
Timestamp: 2023-06-22 22:22:14, TVL: 59304842530.42349
Protocol: Lido
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 14033494583.177282
Protocol: OKX
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 10339938987.89597
Protocol: Bitfinex
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 9560565482.990574
Protocol: MakerDAO
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 6275315042.106567
Protocol: Polygon Bridge & Staking
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 5605853021.235294
Protocol: WBTC
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 4738635119.926148
Protocol: JustLend
Historical TVLs:
Timestamp: 2023-06-22 22:22:15, TVL: 3829177344.3675985
Protocol: Curve DEX
Historical TVLs:
Timestamp: 2023-06-22 22:22:16, TVL: 3808599430.5611534
Protocol: AAVE V2
Historical TVLs:
Timestamp: 2023-06-22 22:22:16, TVL: 3723861501.805204
Protocol: JustCryptos
Historical TVLs:
Timestamp: 2023-06-22 22:22:16, TVL: 35613

Protocol: BendDAO Lending
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 119215946.6554716
Protocol: Firi
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 118083016.7231498
Protocol: cBridge
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 117472098.78621359
Protocol: Acala LCDOT
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 117204536.38632938
Protocol: BiSwap
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 116922157.44960968
Protocol: Pendle
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 112540531.8720411
Protocol: Balancer V1
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 109657573.98705842
Protocol: DefiChain Loans
Historical TVLs:
Timestamp: 2023-06-22 22:22:33, TVL: 108936292.3289617
Protocol: Marinade Finance
Historical TVLs:
Timestamp: 2023-06-22 22:22:34, TVL: 107152216.81523769
Protocol: Kava Lend
Historical TVLs:
Timestamp: 2023-06-22 22:22:34, TVL: 106578158.0303899
Protocol: Cake DeFi
Historical TVLs:
Timestamp: 2023-06-22 22:22:

Protocol: THENA FUSION
Historical TVLs:
Timestamp: 2023-06-22 22:22:54, TVL: 37756756.711135834
Protocol: Idle
Historical TVLs:
Timestamp: 2023-06-22 22:22:54, TVL: 36640585.418475874
Protocol: DODO
Historical TVLs:
Timestamp: 2023-06-22 22:22:54, TVL: 35550523.51231778
Protocol: Orca
Historical TVLs:
Timestamp: 2023-06-22 22:22:54, TVL: 35388170.17528143
Protocol: JPEG'd
Historical TVLs:
Timestamp: 2023-06-22 22:22:55, TVL: 35275116.26557005
Protocol: Origin Ether
Historical TVLs:
Timestamp: 2023-06-22 22:22:55, TVL: 34973757.683378845
Protocol: pNetwork
Historical TVLs:
Timestamp: 2023-06-22 22:22:55, TVL: 34919182.930151515
Protocol: Moonwell Artemis
Historical TVLs:
Timestamp: 2023-06-22 22:22:55, TVL: 34656479.275861
Protocol: Connext
Historical TVLs:
Timestamp: 2023-06-22 22:22:56, TVL: 34408217.571474545
Protocol: Ferro
Historical TVLs:
Timestamp: 2023-06-22 22:22:56, TVL: 34017562.74311969
Protocol: Unslashed
Historical TVLs:
Timestamp: 2023-06-22 22:22:56, TVL: 33794201.197654

Protocol: tBTC
Historical TVLs:
Timestamp: 2023-06-22 22:23:21, TVL: 21401237.1400704
Protocol: Toros
Historical TVLs:
Timestamp: 2023-06-22 22:23:22, TVL: 20876355.782477282
Protocol: RenVM
Historical TVLs:
Timestamp: 2023-06-22 22:23:22, TVL: 20799503.276247364
Protocol: Iron Bank
Historical TVLs:
Timestamp: 2023-06-22 22:23:22, TVL: 20525986.12331271
Protocol: Opyn Gamma
Historical TVLs:
Timestamp: 2023-06-22 22:23:22, TVL: 20505079.120104086
Protocol: Bancor V2.1
Historical TVLs:
Timestamp: 2023-06-22 22:23:23, TVL: 20412131.187182672
Protocol: Sovryn Dex
Historical TVLs:
Timestamp: 2023-06-22 22:23:23, TVL: 19958454.77
Protocol: Exactly
Historical TVLs:
Timestamp: 2023-06-22 22:23:23, TVL: 19770400.76822668
Protocol: ioTube
Historical TVLs:
Timestamp: 2023-06-22 22:23:24, TVL: 19724953.535791427
Protocol: Rhino.fi
Historical TVLs:
Timestamp: 2023-06-22 22:23:24, TVL: 19536710.627509315
Protocol: Equilibria
Historical TVLs:
Timestamp: 2023-06-22 22:23:24, TVL: 19505854.75564517
Pro

Protocol: Jito
Historical TVLs:
Timestamp: 2023-06-22 22:23:56, TVL: 12138666.5381225
Protocol: Lodestar Finance
Historical TVLs:
Timestamp: 2023-06-22 22:23:56, TVL: 11928338.717848657
Protocol: NodeDAO
Historical TVLs:
Timestamp: 2023-06-22 22:23:57, TVL: 11906223.170387322
Protocol: PulseX V2
Historical TVLs:
Timestamp: 2023-06-22 22:23:57, TVL: 11868404.830866968
Protocol: WEMIX.FI AMM
Historical TVLs:
Timestamp: 2023-06-22 22:23:57, TVL: 11626227.7999438
Protocol: Francium
Historical TVLs:
Timestamp: 2023-06-22 22:23:57, TVL: 11522252.262975158
Protocol: Djed Stablecoin
Historical TVLs:
Timestamp: 2023-06-22 22:23:57, TVL: 11474676.412506046
Protocol: Tranchess Ether
Historical TVLs:
Timestamp: 2023-06-22 22:23:58, TVL: 11443854.527731704
Protocol: Pact
Historical TVLs:
Timestamp: 2023-06-22 22:23:58, TVL: 11374435.25354668
Protocol: NestFi
Historical TVLs:
Timestamp: 2023-06-22 22:23:58, TVL: 11364050.267958684
Protocol: LiquidSwap
Historical TVLs:
Timestamp: 2023-06-22 22:23:58,

...

Protocol: Revert Compoundor
Historical TVLs:
Timestamp: 2023-06-22 22:25:00, TVL: 3050202.0145150265
Protocol: WhaleEx
Historical TVLs:
Timestamp: 2023-06-22 22:25:00, TVL: 3041822.4283958147
Protocol: Optim Finance
Historical TVLs:
Timestamp: 2023-06-22 22:25:00, TVL: 3035144.1388032073
Protocol: Mars Ecosystem
Historical TVLs:
Timestamp: 2023-06-22 22:25:01, TVL: 3012032.226092445
Protocol: Waterfall DeFi
Historical TVLs:
Timestamp: 2023-06-22 22:25:01, TVL: 3011934.851201218
Protocol: WINR Protocol
Historical TVLs:
Timestamp: 2023-06-22 22:25:01, TVL: 2993555.472902866
Protocol: Ceres Demeter
Historical TVLs:
Timestamp: 2023-06-22 22:25:01, TVL: 2979818.639559212
Protocol: Extra Finance
Historical TVLs:
Timestamp: 2023-06-22 22:25:01, TVL: 2977240.760957039
Protocol: GrizzlyFi
Historical TVLs:
Timestamp: 2023-06-22 22:25:02, TVL: 2972722.3783295946
Protocol: Trisolaris
Historical TVLs:
Timestamp: 2023-06-22 22:25:02, TVL: 2967806.405092135
Protocol: FLRX
Historical TVLs:
Timestamp: 

Protocol: Perlin
Historical TVLs:
Timestamp: 2023-06-22 22:25:18, TVL: 2201525.5016107843
Protocol: Humble Defi
Historical TVLs:
Timestamp: 2023-06-22 22:25:18, TVL: 2196872.822703
Protocol: Bluefin
Historical TVLs:
Timestamp: 2023-06-22 22:25:18, TVL: 2193268.25951
Protocol: Unipilot
Historical TVLs:
Timestamp: 2023-06-22 22:25:18, TVL: 2183487.3645382295
Protocol: Unicly
Historical TVLs:
Timestamp: 2023-06-22 22:25:18, TVL: 2182016.600182649
Protocol: Morphex
Historical TVLs:
Timestamp: 2023-06-22 22:25:18, TVL: 2177124.837679817
Protocol: El Dorado Exchange
Historical TVLs:
Timestamp: 2023-06-22 22:25:19, TVL: 2173557.13393903
Protocol: Quicksilver Protocol
Historical TVLs:
Timestamp: 2023-06-22 22:25:19, TVL: 2172852.4737933814
Protocol: Parallel AMM
Historical TVLs:
Timestamp: 2023-06-22 22:25:19, TVL: 2160645.39054946
Protocol: Thales
Historical TVLs:
Timestamp: 2023-06-22 22:25:19, TVL: 2148296.0458416273
Protocol: sTLOS Liquid Staking
Historical TVLs:
Timestamp: 2023-06-22 22:2

Protocol: Hydradex V3
Historical TVLs:
Timestamp: 2023-06-22 22:25:35, TVL: 1488079.0742668423
Protocol: NerveSwap
Historical TVLs:
Timestamp: 2023-06-22 22:25:35, TVL: 1485159.521225
Protocol: Bao Masterchef V1
Historical TVLs:
Timestamp: 2023-06-22 22:25:35, TVL: 1483153.6262352383
Protocol: Prime Protocol
Historical TVLs:
Timestamp: 2023-06-22 22:25:36, TVL: 1467569.8567176447
Protocol: Arbswap StableSwap
Historical TVLs:
Timestamp: 2023-06-22 22:25:36, TVL: 1462918.6866330532
Protocol: Demex
Historical TVLs:
Timestamp: 2023-06-22 22:25:36, TVL: 1461750.444612264
Protocol: Maia V3
Historical TVLs:
Timestamp: 2023-06-22 22:25:36, TVL: 1459104.6296655803
Protocol: handle.fi Perps
Historical TVLs:
Timestamp: 2023-06-22 22:25:36, TVL: 1451039.0606374901
Protocol: Nested
Historical TVLs:
Timestamp: 2023-06-22 22:25:37, TVL: 1446449.620126246
Protocol: Bunicorn
Historical TVLs:
Timestamp: 2023-06-22 22:25:37, TVL: 1443958.257939
Protocol: PieDAO
Historical TVLs:
Timestamp: 2023-06-22 22:2

Protocol: NF3 APE
Historical TVLs:
Timestamp: 2023-06-22 22:25:55, TVL: 1034617.0083629854
Protocol: Tenderize
Historical TVLs:
Timestamp: 2023-06-22 22:25:55, TVL: 1031425.0963641017
Protocol: nftperp
Historical TVLs:
Timestamp: 2023-06-22 22:25:56, TVL: 1025999.8221595834
Protocol: Parcl
Historical TVLs:
Timestamp: 2023-06-22 22:25:56, TVL: 1018852.450642
Protocol: Taiga Protocol
Historical TVLs:
Timestamp: 2023-06-22 22:25:56, TVL: 1016209.2738111425
Protocol: Sphynx
Historical TVLs:
Timestamp: 2023-06-22 22:25:56, TVL: 1013817.2000295604
Protocol: Monopoly Finance
Historical TVLs:
Timestamp: 2023-06-22 22:25:56, TVL: 1011127.9909649218
Protocol: Argo Finance
Historical TVLs:
Timestamp: 2023-06-22 22:25:57, TVL: 1011073.0009541688
Protocol: Kintsugi
Historical TVLs:
Timestamp: 2023-06-22 22:25:57, TVL: 1009394.9717100401
Protocol: Bond Protocol
Historical TVLs:
Timestamp: 2023-06-22 22:25:57, TVL: 999772.7082631025
Protocol: MM Finance Arbitrum V3
Historical TVLs:
Timestamp: 2023-06

Protocol: Tethys Perpetual
Historical TVLs:
Timestamp: 2023-06-22 22:26:16, TVL: 743039.7352907312
Protocol: Tegisto
Historical TVLs:
Timestamp: 2023-06-22 22:26:17, TVL: 739985.309552657
Protocol: Flair Dex
Historical TVLs:
Timestamp: 2023-06-22 22:26:17, TVL: 737907.4927186004
Protocol: Vendor V1
Historical TVLs:
Timestamp: 2023-06-22 22:26:17, TVL: 734432.9938132159
Protocol: TangoSwap
Historical TVLs:
Timestamp: 2023-06-22 22:26:17, TVL: 734060.1155417692
Protocol: Iron Finance
Historical TVLs:
Timestamp: 2023-06-22 22:26:17, TVL: 730597.8539332687
Protocol: Astroport Classic
Historical TVLs:
Timestamp: 2023-06-22 22:26:18, TVL: 727835.9095849658
Protocol: LuaSwap
Historical TVLs:
Timestamp: 2023-06-22 22:26:18, TVL: 726666.8888514357
Protocol: Lif3 Trade
Historical TVLs:
Timestamp: 2023-06-22 22:26:18, TVL: 724841.7352751533
Protocol: Sharky
Historical TVLs:
Timestamp: 2023-06-22 22:26:18, TVL: 723462.6603842899
Protocol: Grim Finance
Historical TVLs:
Timestamp: 2023-06-22 22:26:1